In [ ]:
! pip install -r requirements.txt

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM

#model_id = "google/gemma-2b-it"

model_id = "distilbert/distilgpt2"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16)
#model.to_bettertransformer()

model.eval()

device = 'cpu' 

In [4]:
def inference(question: str, context: str):

    if context == None or context == "":
        prompt = f"""Give a detailed answer to the following question. Question: {question}"""
    else:
        prompt = f"""Using the information contained in the context, give a detailed answer to the question.
            Context: {context}.
            Question: {question}"""
    chat = [
        {"role": "user", "content": prompt}
    ]
    formatted_prompt = tokenizer.apply_chat_template(
        chat,
        tokenize=False,
        add_generation_prompt=True,
    )
    inputs = tokenizer.encode(
        formatted_prompt, add_special_tokens=False, return_tensors="pt"
    ).to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs,
            max_new_tokens=250,
            do_sample=False,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    response = response[len(formatted_prompt) :]  # remove input prompt from reponse
    response = response.replace("<eos>", "")  # remove eos token
    return response


question = "What is a transformer?"
print(inference(question=question, context=""))


No chat template is defined for this tokenizer - using the default template for the GPT2TokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


The first of a series of articles on the subject of the "The Biggest Secret" of the Internet is a new book about the Internet.



The Biggest Secret of the Internet is a book about the Internet.


The Biggest Secret of the Internet is a book about the Internet.

The Biggest Secret of the Internet is a book about the Internet.
The Biggest Secret of the Internet is a book about the Internet.
The Biggest Secret of the Internet is a book about the Internet.
The Biggest Secret of the Internet is a book about the Internet.
The Biggest Secret of the Internet is a book about the Internet.
The Biggest Secret of the Internet is a book about the Internet.
The Biggest Secret of the Internet is a book about the Internet.
The Biggest Secret of the Internet is a book about the Internet.
The Biggest Secret of the Internet is a book about the Internet.
The Biggest Secret of the Internet is a book about the Internet.
The Biggest Secret of the Internet is a book about the Internet.
The Biggest Secret of 

In [14]:
def infer(question: str, context: str):

    if context == None or context == "":
        prompt = f"""Give a detailed answer to the following question. Question: {question}"""
    else:
        prompt = f"""Using the information contained in the context, give a detailed answer to the question.
            Context: {context}.
            Question: {question}"""    
    response = ollama.generate(model='gemma:2b', prompt=prompt)
   
    return response

In [5]:
import ollama

ollama.generate(model='gemma:2b', prompt=question)

{'model': 'gemma:2b',
 'created_at': '2024-05-22T02:41:02.0002534Z',
 'response': 'A transformer is an electrical device that transfers energy from one circuit to another through induction. It is a passive device, meaning that it does not generate or consume electrical energy itself.\n\n**Key characteristics of a transformer:**\n\n* **Inductance:** The ability of a transformer to store and release energy in an alternating current circuit.\n* **Voltage:** The amount of voltage supplied to the primary circuit.\n* **Current:** The amount of current flowing through the primary and secondary circuits.\n* **Power:** The rate at which energy is transferred from the primary to the secondary circuit.\n* **Turns ratio:** The ratio of the number of turns in the primary to the number of turns in the secondary circuit.\n\n**How a transformer works:**\n\nA transformer works based on the principle of electromagnetic induction. When an alternating current is applied to the primary circuit, it creates 

In [7]:
from langchain.document_loaders import PyPDFLoader

loaders = [
    PyPDFLoader("files/LLMclinical.pdf")
]
pages = []
for loader in loaders:
    pages.extend(loader.load())

In [8]:
from langchain.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(chunk_size=128, chunk_overlap=12)
docs = text_splitter.split_documents(pages)

In [10]:
print(docs[2].page_content)

elle Barral1, 
Christopher Semturs1, Alan Karthikesalingam1,5 ✉ & Vivek Natarajan1,5 ✉
Large language models (LLMs) have demonstrated impressive capabilities, but the 
bar for clinical applications is high. Attempts to assess the clinical knowledge of 
models typically rely on automated evaluations based on limited benchmarks. Here, to address these limitations, we present MultiMedQA, a benchmark combining six 
existing medical question answering datasets spanning professional medicine, 
research and consumer queries and a new dataset of medical questions


In [11]:
import numpy as np
from langchain_community.embeddings import (
    HuggingFaceEmbeddings
)
encoder = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L12-v2', model_kwargs = {'device': "cpu"})

In [9]:
embeddings1 = encoder.embed_query("RAG")
embeddings2 = encoder.embed_query(docs[0].page_content)
print(np.dot(embeddings1, embeddings2))

0.02857219219547718


In [10]:
from langchain.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy

faiss_db = FAISS.from_documents(docs, encoder, distance_strategy=DistanceStrategy.DOT_PRODUCT)

In [12]:
question = "What does COT prompting mean?"
retrieved_docs = faiss_db.similarity_search(question, k=5)
context = "".join(doc.page_content + "\n" for doc in retrieved_docs)
print(context)

 the abstract context within the prompt text.
Chain-of-thought prompting. COT16 involves augmenting each 
few-shot example in the prompt with a step-by-step breakdown and 
a coherent set of intermediate reasoning steps towards the final  
answer. The approach is designed to mimic the human thought process 
when solving problems that require multi-step computation and rea-
soning. COT prompting can elicit reasoning abilities in sufficiently LLMs 
and dramatically improve performance on tasks such as mathe  matical 
Articleproblems16,62. Further, the appearance of such COT reasoning  
appears to be an emergent ability28 of LLMs. COT prompting has been 
used to achieve breakthrough LLM performance on several STEM  
benchmarks63.
Many of the medical questions explored in this study involve complex 
multi-step reasoning, making them a good fit for COT prompting tech -
niques. Together with clinicians, we crafted COT prompts to provide 
clear demonstrations on how to reason and answer the gi

In [ ]:
torch.cuda.empty_cache()
print(inference(question=question, context=context))

In [15]:
infer(question=question, context=context)

{'model': 'gemma:2b',
 'created_at': '2024-05-17T07:08:51.8497022Z',
 'response': 'The context does not provide a detailed definition of what COT prompting means, so I cannot answer this question from the provided context.',
 'done': True,
 'context': [106,
  1645,
  108,
  15325,
  573,
  2113,
  13614,
  575,
  573,
  4807,
  235269,
  2734,
  476,
  11352,
  3448,
  577,
  573,
  2872,
  235265,
  108,
  149,
  2930,
  235292,
  139,
  1175,
  9949,
  4807,
  2819,
  573,
  18335,
  2793,
  235265,
  108,
  25482,
  235290,
  559,
  235290,
  47185,
  103419,
  235265,
  91593,
  235274,
  235318,
  18348,
  35126,
  574,
  1853,
  235248,
  108,
  53257,
  235290,
  23902,
  3287,
  575,
  573,
  18335,
  675,
  476,
  4065,
  235290,
  1547,
  235290,
  8847,
  25497,
  578,
  235248,
  108,
  235250,
  63269,
  1142,
  576,
  26840,
  32346,
  7161,
  6643,
  573,
  2048,
  139,
  108,
  13072,
  235265,
  714,
  5688,
  603,
  6869,
  577,
  78763,
  573,
  3515,
  3421,
  2185,

In [ ]:
print("For this answer I used the following documents:")
for doc in retrieved_docs:
    print(doc.metadata)